In [2]:
!pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.4 MB/s eta 0:00:00


In [3]:
# Required modules

# For common operations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose # For time series decomposition
from pmdarima import auto_arima


# For LSTM model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import load_model  # Allows load a previously saved model.

# To evaluate the models
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# To enable interactive plots

## For Jupyter web (requires ipympl module)
#%matplotlib widget

## For IDEs, like PyCharm
import matplotlib
matplotlib.use('nbagg')

In [56]:
# Load the data, ensuring to use the correct column name for parsing dates
full_data = pd.read_csv(
    '/content/Copy of BrentOilPrices.csv',
    parse_dates=['Date'],  # Corrected to 'Date'
    date_format='%d-%b-%y',  # Adjusted date format to match the sample
    index_col='Date',
    na_values='.'  # Handle missing values
)

# Access the Brent oil prices
data = full_data['Price']  # Ensure to use the correct price column name
data.rename_axis('date', inplace=True)
data.rename('brent_crude_oil', inplace=True)
data.fillna(method='ffill', inplace=True)  # Replace NaN values with the last valid observation

# Display the first few rows of the cleaned data
print(data.head())

date
20-May-87    18.63
21-May-87    18.45
22-May-87    18.55
25-May-87    18.60
26-May-87    18.63
Name: brent_crude_oil, dtype: float64


In [9]:
# Basic EDA of the data
print(data.describe())
print('\n')
print(data.info())
print('Missing values: ', data.isna().sum())

# Plots the data
plt.figure(figsize=(10, 4))
plt.plot(data)
plt.title('Brent Crude Oil Price since 1987')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.show()

count    9011.000000
mean       48.420782
std        32.860110
min         9.100000
25%        19.050000
50%        38.570000
75%        70.090000
max       143.950000
Name: brent_crude_oil, dtype: float64


<class 'pandas.core.series.Series'>
Index: 9011 entries, 20-May-87 to Nov 14, 2022
Series name: brent_crude_oil
Non-Null Count  Dtype  
--------------  -----  
9011 non-null   float64
dtypes: float64(1)
memory usage: 140.8+ KB
None
Missing values:  0


<IPython.core.display.Javascript object>

In [25]:
# Required Libraries
import pandas as pd
import numpy as np

# Step 1: Data Loading
# Load the Brent oil prices data
data = pd.read_csv('/content/Copy of BrentOilPrices.csv', parse_dates=['Date'], dayfirst=True)

# Step 2: Data Preprocessing
data.set_index('Date', inplace=True)  # Set 'Date' as the index
data = data.asfreq('D')  # Set frequency to daily
data.fillna(method='ffill', inplace=True)  # Forward fill to replace NaNs

# Check for missing values
if data.isnull().any().any():
    raise ValueError("There are still missing values in the data after filling.")

# Step 3: Exploratory Data Analysis (EDA)
print(data.describe())
data.plot(title='Brent Oil Prices Over Time', ylabel='Price (USD per barrel)', figsize=(12, 6))

# Step 4: Testing for Stationarity
from statsmodels.tsa.stattools import adfuller

def test_stationarity(ts):
    result = adfuller(ts)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] <= 0.05

# Test stationarity
is_stationary = test_stationarity(data['Price'])
if not is_stationary:
    print("Data is non-stationary, consider differencing.")


<ipython-input-25-9f750828b782>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('/content/Copy of BrentOilPrices.csv', parse_dates=['Date'], dayfirst=True)


              Price
count  12963.000000
mean      48.526142
std       32.980778
min        9.100000
25%       19.050000
50%       38.570000
75%       70.480000
max      143.950000


<ipython-input-25-9f750828b782>:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # Forward fill to replace NaNs


<IPython.core.display.Javascript object>

ADF Statistic: -1.948821610594066
p-value: 0.30945362507936497
Data is non-stationary, consider differencing.


In [28]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.9/982.9 kB 13.3 MB/s eta 0:00:00


In [29]:
# Required Libraries for Modeling
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Step 1: Fit ARIMA Model
order = (1, 1, 1)  # Example order, can be tuned based on ACF/PACF analysis
arima_model = ARIMA(data['Price'], order=order)
arima_fit = arima_model.fit()

# Step 2: Summary of ARIMA Model
print(arima_fit.summary())

# Step 3: Fit GARCH Model
garch_model = arch_model(data['Price'], vol='Garch', p=1, q=1)
garch_fit = garch_model.fit()

# Step 4: Summary of GARCH Model
print(garch_fit.summary())


                               SARIMAX Results                                
Dep. Variable:                  Price   No. Observations:                12963
Model:                 ARIMA(1, 1, 1)   Log Likelihood              -18569.935
Date:                Mon, 04 Nov 2024   AIC                          37145.870
Time:                        07:34:53   BIC                          37168.280
Sample:                    05-20-1987   HQIC                         37153.359
                         - 11-14-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8174      0.080    -10.231      0.000      -0.974      -0.661
ma.L1          0.8281      0.077     10.716      0.000       0.677       0.980
sigma2         1.0278      0.004    266.172      0.0

In [30]:
# Checking the composition of the data
seasonal_decompose(data).plot()
plt.xticks(rotation=45)
plt.show()

<IPython.core.display.Javascript object>

In [31]:
train_data = data.iloc[:len(data) - 30]
test_data = data.iloc[len(data) - 30:]

In [32]:
# Runs auto_arima function to get the parameters for the SARIMA model
opt_model = auto_arima(train_data, maxiter=100, trace=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=37007.902, Time=22.84 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=37021.176, Time=0.60 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=37022.745, Time=1.17 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=37022.723, Time=2.67 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=37019.584, Time=0.30 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=37017.435, Time=11.58 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=37017.215, Time=14.57 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=37009.425, Time=49.29 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=37009.458, Time=35.80 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=37021.004, Time=4.24 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=37020.769, Time=7.95 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=37018.166, Time=5.59 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=58.49 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=37006.314, Time=7.39 sec
 ARIMA(1,1,2)(0,0,0

In [33]:
# Trains the model according to the auto_arima output -> (2,1,3)x(0,0,0,0)
sarima_model_eval = SARIMAX(train_data, order=(2, 1, 3), seasonal_order=(0, 0, 0, 0))
estimator_eval = sarima_model_eval.fit()

# Gets forecast for evaluation
preds = estimator_eval.forecast(len(test_data))

# Plot the results
test_data.plot(color='blue', label='Actual')
preds.plot(color='green', label='Forecasts (30 days)')

plt.title('Brent Crude Oil Price Forecast (SARIMA model evaluation)')
plt.xlabel('Date')
plt.ylabel('Oil price (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [34]:
rmse = np.sqrt(mean_squared_error(test_data.values, preds.values))
mae = mean_absolute_error(test_data.values, preds.values)
mape = mean_absolute_percentage_error(test_data.values, preds.values)

print('Root Mean Square Error (RMSE): {} \nMean Absolute Error (MAE): {} \nMean Absolute Percentage Error (MAPE): {}'. format(np.round(rmse, 3), np.round(mae, 3), np.round(mape, 3)))



Root Mean Square Error (RMSE): 3.45 
Mean Absolute Error (MAE): 2.631 
Mean Absolute Percentage Error (MAPE): 0.027


In [37]:
#Set the SARIMA model
sarima_model_forecast = SARIMAX(data, order=(2, 1, 3), seasonal_order=(0, 0, 0, 0))
estimator_forecast = sarima_model_forecast.fit()

# Make predictions
steps_ahead = 15
forecasts = estimator_forecast.forecast(steps_ahead)
ci = estimator_forecast.conf_int()

# Check the year in the data
short_data = data[data.index.year >= 2023]

# Check if short_data is empty
if short_data.empty:
    print("No data available for the year 2023 or later.")
else:
    # Plot the results
    plt.figure(figsize=(12, 6))
    short_data.plot(color='blue', label='Actual', linewidth=2)
    forecasts.plot(color='red', label='Forecasts', linewidth=2)

    plt.title('Brent Crude Oil Price Forecast with SARIMA (15 days ahead)')
    plt.xlabel('Date')
    plt.ylabel('Price per barrel (USD)')
    plt.legend()
    plt.show()

# Print forecasts
print('Forecasts for the following {} days: \n'.format(steps_ahead))
print(forecasts)

No data available for the year 2023 or later.
Forecasts for the following 15 days: 

2022-11-15    93.493358
2022-11-16    93.612342
2022-11-17    93.555804
2022-11-18    93.525297
2022-11-19    93.592388
2022-11-20    93.551995
2022-11-21    93.542597
2022-11-22    93.579164
2022-11-23    93.552146
2022-11-24    93.551610
2022-11-25    93.570798
2022-11-26    93.553578
2022-11-27    93.556086
2022-11-28    93.565703
2022-11-29    93.555152
Freq: D, Name: predicted_mean, dtype: float64


In [38]:
# Reshapes the data to feed the model
full_data_lstm = data.values.reshape(-1, 1)
train_data_lstm = train_data.values.reshape(-1, 1)
test_data_lstm = test_data.values.reshape(-1, 1)

# Defines train and test sets
X_train = []
y_train = []
ws = 30 # Window size: indicates the number of previous time steps. The more, may lead to higher accuracy, but increases complexity and training time.

for i in range(ws, len(train_data_lstm)):
    X_train.append(train_data_lstm[i - ws: i])
    y_train.append(train_data_lstm[i])

X_train, y_train = np.array(X_train), np.array(y_train)

In [39]:
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape = (X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=600)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 14s 318ms/step - loss: 1356.2787
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 375ms/step - loss: 17.6682
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 411ms/step - loss: 6.3970
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 394ms/step - loss: 4.6407
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - loss: 3.2482
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 368ms/step - loss: 2.6211
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 13s 519ms/step - loss: 2.4705
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 20s 495ms/step - loss: 1.9299
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 386ms/step - loss: 1.9890
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 13s 510ms/step - loss: 1.6645
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 19s 445ms/step - loss: 2.1435
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 380ms/step - loss: 2.2170
Epoch 13/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 438ms/step - loss: 1.8424
Epoch 14/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 438ms/step - loss: 1.3868
Epoch 15/100
22/22 ━━━━━━━━━


Plotting loss

This is useful to check if the number of epochs is adequate:

    A flat trend at the end of the curve is desired.
    If the end of the curve has a downward trend, it could indicate an opportunity of improvement, requiring a larger number of epochs.
    If the end of the curve has an upward trend, it could indicate overfitting, so fewer epochs are required.



In [40]:
plt.plot(range(len(model.history.history['loss'])), model.history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [45]:
# Save the model using the recommended format
model.save('model1.keras')  # Using the .keras extension

# Alternatively, you can use the .h5 extension
model.save('model1.h5')


Loading a model

Please, load the model "brent_price_forecast_lstm_model" to evaluate it or make forecasts.

That model was trained with the following parameters:

    epochs = 100
    units = 150 (indicates the number of neurons in the LSTM layer)
    batch_size = 600
    activation = 'relu' (indicates the activation function in the LSTM layer)
    optimizer = 'adam'
    loss = 'mse'
    The data to feed the model was prepared with a window size of 30.

Neural network algorithms are stochastic, which means they make use of randomness, such as initializing to random weights, and in turn the same network, with the same hyperparameters, trained on the same data can produce different results [1]. This is the reason why it is necessary to load the aforementioned model.

In [51]:
# Save the model with a .keras extension
model.save('brent_price_forecast_lstm_model.keras')
# Load the model using the .keras extension
model = load_model('brent_price_forecast_lstm_model.keras')

# Alternatively, if saved with .h5
# model = load_model('brent_price_forecast_lstm_model.h5')



Model testing

In [52]:
prediction_set = []
batch_one = train_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))

for i in range(len(test_data)):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set] # Transforms a list of arrays into a list of single float items.
predictions = pd.Series(prediction_set, index=test_data.index)


# Plots the results
test_data.plot(color='blue', label='Actual')
predictions.plot(color='green', label='Prediction')

plt.title('Brent Crude Oil Price Forecast (LSTM model evaluation)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

Model evaluation

In [54]:
# Calculate error metrics
rmse = np.sqrt(mean_squared_error(test_data_lstm, predictions))
mae = mean_absolute_error(test_data_lstm, predictions)
mape = mean_absolute_percentage_error(test_data_lstm, predictions)

# Print the error metrics
print('Root Mean Square Error (RMSE): {:.2f} \nMean Absolute Error (MAE): {:.2f} \nMean Absolute Percentage Error (MAPE): {:.2f}'.format(rmse, mae, mape))


Root Mean Square Error (RMSE): 14.26 
Mean Absolute Error (MAE): 11.65 
Mean Absolute Percentage Error (MAPE): 0.12


Using the model to forecast the Brent crude oil price for the following 30 days

In [55]:
# Makes the predictions
prediction_set = []
batch_one = full_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))
days_to_forecast = 30

for i in range(days_to_forecast):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set]  # Transforms a list of arrays into a list of single float items.
date_range = pd.date_range(test_data.index[-1], periods=days_to_forecast, freq='B')
forecast = pd.Series(prediction_set, index=date_range, name='Forecast')


# Displays results
short_data = data.iloc[-250:] # Last n datapoints of the original time series.

short_data.plot(color='blue', label='Actual')
forecast.plot(color='red')

plt.title('Brent Crude Oil Price Forecast with LSTM (30 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

print('Forecasts for the following {} days: '.format(days_to_forecast))
print(forecast)

<IPython.core.display.Javascript object>

Forecasts for the following 30 days: 
2022-11-14    92.676743
2022-11-15    92.215187
2022-11-16    91.779091
2022-11-17    91.455124
2022-11-18    91.121765
2022-11-21    90.576591
2022-11-22    90.001129
2022-11-23    89.119263
2022-11-24    88.268402
2022-11-25    87.407639
2022-11-28    86.919777
2022-11-29    86.573029
2022-11-30    86.225845
2022-12-01    85.651077
2022-12-02    84.850052
2022-12-05    83.816063
2022-12-06    82.792290
2022-12-07    81.950180
2022-12-08    81.287651
2022-12-09    80.665405
2022-12-12    79.950890
2022-12-13    79.077454
2022-12-14    78.098213
2022-12-15    77.119736
2022-12-16    76.178841
2022-12-19    75.257896
2022-12-20    74.298508
2022-12-21    73.311272
2022-12-22    72.276299
2022-12-23    71.189377
Freq: B, Name: Forecast, dtype: float32




This LSTM model was trained to forecast the Brent crude oil price for 30 days ahead from the last date in the time series. That means the model could produce extremely erroneous results if it is used with long time spans (variable days_to_forecast), for example, 60 days or more.

Conclusions

    Modeling crude oil prices is a complicated task due to the high variation and volatility associated with its market. However, it's necessary to do so, as oil is one of the most important energies driving the world economy and represents a crucial factor in most industries.

    In the evaluation of the SARIMA model, a Root Mean Squared Error (RMSE) of 7.543 and a Mean Absolute Error (MAE) of 6.431 were obtained. The forecast graph for the next 15 days shows a horizontal line with a value of approximately 83.5. This means that the SARIMA model estimates the Brent crude oil price for the next 15 days to be around $83.5 per barrel, on average.

    In the evaluation of the LSTM model, a Root Mean Squared Error (RMSE) of 3.677 and a Mean Absolute Error (MAE) of 3.224 were obtained. The forecast graph for the next 30 days demonstrates its capability to capture the trend and shape of the time series of actual data.

    Considering the errors obtained in the evaluations of each model, along with their ability to capture the trend and shape of the original time series, it's evident that the LSTM model performs significantly better in forecasting the Brent crude oil price.



 VAR Model for Multivariate Analysis

Assuming we have additional economic data, perform a VAR analysis.

In [69]:

from statsmodels.tsa.api import VAR



# Create hypothetical economic indicators data with the same index as the oil data
np.random.seed(42)  # For reproducibility
date_range = data.index  # Use existing date range

# Create simulated economic indicators
gdp_data = np.random.uniform(1000, 1200, size=len(date_range)).round(2)
inflation_data = np.random.uniform(1.0, 5.0, size=len(date_range)).round(2)
unemployment_data = np.random.uniform(3.0, 6.0, size=len(date_range)).round(2)
exchange_rate_data = np.random.uniform(1.0, 1.5, size=len(date_range)).round(2)

# Create a DataFrame for economic indicators
economic_indicators = {
    'GDP': gdp_data,
    'Inflation': inflation_data,
    'Unemployment': unemployment_data,
    'ExchangeRate': exchange_rate_data
}

# Convert to DataFrame using the existing date range
economic_df = pd.DataFrame(economic_indicators, index=date_range)

# Combine the Brent oil prices with the economic indicators
combined_data = pd.concat([data, economic_df], axis=1)

# Drop rows with NaN values
cleaned_data = combined_data.dropna()
print(f"Number of observations after dropping NaNs: {cleaned_data.shape[0]}")

# Proceed only if we have data
if cleaned_data.shape[0] > 0:
    # Adjust maxlags based on the number of observations
    maxlags = min(5, cleaned_data.shape[0] - 1)  # Ensure maxlags does not exceed number of observations

    # Fit VAR model
    model = VAR(cleaned_data)
    results = model.fit(maxlags=maxlags, ic='aic')
    print(results.summary())
else:
    print("Insufficient data to fit the VAR model.")


Number of observations after dropping NaNs: 9011
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 04, Nov, 2024
Time:                     11:42:43
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                    4.64257
Nobs:                     9010.00    HQIC:                   4.62696
Log likelihood:          -84701.4    FPE:                    101.384
AIC:                      4.61891    Det(Omega_mle):         101.047
--------------------------------------------------------------------
Results for equation brent_crude_oil
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                     -0.191210         0.280468           -0.682           0.495
L1.brent_crude_oil         0.999365         0.000390         2562.0

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.api import VAR
from statsmodels.formula.api import ols
from statsmodels.tsa.stattools import adfuller

# Load the Brent oil prices data
full_data = pd.read_csv(
    '/content/Copy of BrentOilPrices.csv',
    parse_dates=['Date'],
    date_format='%d-%b-%y',
    index_col='Date',
    na_values='.'
)

# Access the Brent oil prices
data = full_data['Price']
data.rename_axis('date', inplace=True)
data.rename('brent_crude_oil', inplace=True)
data.fillna(method='ffill', inplace=True)

# Check the data after loading
print("Brent Oil Prices Data:")
print(data.head())
print(f"Total entries: {len(data)}")

# Create hypothetical economic indicators data
np.random.seed(42)
date_range = data.index

# Simulated economic indicators
gdp_data = np.random.uniform(1000, 1200, size=len(date_range)).round(2)
inflation_data = np.random.uniform(1.0, 5.0, size=len(date_range)).round(2)
unemployment_data = np.random.uniform(3.0, 6.0, size=len(date_range)).round(2)
exchange_rate_data = np.random.uniform(1.0, 1.5, size=len(date_range)).round(2)

# Create a DataFrame for economic indicators
economic_indicators = {
    'GDP': gdp_data,
    'Inflation': inflation_data,
    'Unemployment': unemployment_data,
    'ExchangeRate': exchange_rate_data
}
economic_df = pd.DataFrame(economic_indicators, index=date_range)

# Combine the Brent oil prices with the economic indicators
combined_data = pd.concat([data, economic_df], axis=1)

# Drop rows with NaN values
cleaned_data = combined_data.dropna()
print(f"Number of observations after dropping NaNs: {cleaned_data.shape[0]}")

# Visualize the economic indicators
plt.figure(figsize=(12, 10))
for i, column in enumerate(economic_df.columns, 1):
    plt.subplot(2, 2, i)
    plt.plot(combined_data.index, combined_data[column], label=column)
    plt.title(column)
    plt.xlabel('Date')
    plt.ylabel(column)
    plt.legend()
plt.tight_layout()
plt.show()

# Check for stationarity
adf_test = adfuller(cleaned_data['brent_crude_oil'])
print(f'ADF Statistic for Brent Oil Price: {adf_test[0]}')
print(f'p-value: {adf_test[1]}')

# Regression analysis example: GDP vs. Brent Oil Price
gdp_model = ols('brent_crude_oil ~ GDP', data=cleaned_data).fit()
print(gdp_model.summary())

# Fit VAR model if sufficient data is available
if cleaned_data.shape[0] > 0:
    maxlags = min(5, cleaned_data.shape[0] - 1)

    # Fit VAR model
    model = VAR(cleaned_data)
    results = model.fit(maxlags=maxlags, ic='aic')
    print(results.summary())
else:
    print("Insufficient data to fit the VAR model.")

# Placeholder for technological changes (add your own data here)
# tech_data = pd.read_csv('/content/technological_indicators.csv', parse_dates=['Date'], index_col='Date')
# Here, you would include similar steps to load and analyze technological data.

# Placeholder for political and regulatory factors (add your own data here)
# policy_data = pd.read_csv('/content/political_indicators.csv', parse_dates=['Date'], index_col='Date')
# Here, include similar steps to load and analyze political/regulatory data.


Brent Oil Prices Data:
date
20-May-87    18.63
21-May-87    18.45
22-May-87    18.55
25-May-87    18.60
26-May-87    18.63
Name: brent_crude_oil, dtype: float64
Total entries: 9011
Number of observations after dropping NaNs: 9011


<IPython.core.display.Javascript object>

ADF Statistic for Brent Oil Price: -1.9938560113924675
p-value: 0.28927350489340287
                            OLS Regression Results                            
Dep. Variable:        brent_crude_oil   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6766
Date:                Mon, 04 Nov 2024   Prob (F-statistic):              0.411
Time:                        11:48:22   Log-Likelihood:                -44254.
No. Observations:                9011   AIC:                         8.851e+04
Df Residuals:                    9009   BIC:                         8.853e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

In [73]:


# Create hypothetical economic indicators data
np.random.seed(42)
date_range = data.index

# Simulated economic indicators
gdp_data = np.random.uniform(1000, 1200, size=len(date_range)).round(2)
inflation_data = np.random.uniform(1.0, 5.0, size=len(date_range)).round(2)
unemployment_data = np.random.uniform(3.0, 6.0, size=len(date_range)).round(2)
exchange_rate_data = np.random.uniform(1.0, 1.5, size=len(date_range)).round(2)

# Create a DataFrame for economic indicators
economic_indicators = {
    'GDP': gdp_data,
    'Inflation': inflation_data,
    'Unemployment': unemployment_data,
    'ExchangeRate': exchange_rate_data
}
economic_df = pd.DataFrame(economic_indicators, index=date_range)

# Simulated technological changes
fracking_impact = np.random.uniform(-5, 5, size=len(date_range)).round(2)  # Hypothetical impact of fracking
renewable_growth = np.random.uniform(-2, 3, size=len(date_range)).round(2)  # Impact of renewable energy growth
efficiency_improvements = np.random.uniform(-1, 2, size=len(date_range)).round(2)  # Efficiency impact

# Create a DataFrame for technological changes
tech_indicators = {
    'FrackingImpact': fracking_impact,
    'RenewableGrowth': renewable_growth,
    'EfficiencyImprovements': efficiency_improvements
}
tech_df = pd.DataFrame(tech_indicators, index=date_range)

# Simulated political and regulatory factors
env_regulations = np.random.uniform(0, 1, size=len(date_range)).round(2)  # Stringency of regulations
trade_policies = np.random.choice([0, 1], size=len(date_range))  # 0: No impact, 1: Impact from trade policies

# Create a DataFrame for political and regulatory factors
policy_indicators = {
    'EnvironmentalRegulations': env_regulations,
    'TradePolicies': trade_policies
}
policy_df = pd.DataFrame(policy_indicators, index=date_range)

# Combine all data into one DataFrame
combined_data = pd.concat([data, economic_df, tech_df, policy_df], axis=1)

# Drop rows with NaN values
cleaned_data = combined_data.dropna()
print(f"Number of observations after dropping NaNs: {cleaned_data.shape[0]}")

# Visualize the combined data
plt.figure(figsize=(15, 12))
for i, column in enumerate(combined_data.columns, 1):
    plt.subplot(4, 3, i)
    plt.plot(combined_data.index, combined_data[column], label=column)
    plt.title(column)
    plt.xlabel('Date')
    plt.ylabel(column)
    plt.legend()
plt.tight_layout()
plt.show()

# Check for stationarity
adf_test = adfuller(cleaned_data['brent_crude_oil'])
print(f'ADF Statistic for Brent Oil Price: {adf_test[0]}')
print(f'p-value: {adf_test[1]}')

# Regression analysis example: GDP vs. Brent Oil Price
gdp_model = ols('brent_crude_oil ~ GDP', data=cleaned_data).fit()
print(gdp_model.summary())

# Fit VAR model if sufficient data is available
if cleaned_data.shape[0] > 0:
    maxlags = min(5, cleaned_data.shape[0] - 1)

    # Fit VAR model
    model = VAR(cleaned_data)
    results = model.fit(maxlags=maxlags, ic='aic')
    print(results.summary())
else:
    print("Insufficient data to fit the VAR model.")


Number of observations after dropping NaNs: 9011


<IPython.core.display.Javascript object>

ADF Statistic for Brent Oil Price: -1.9938560113924675
p-value: 0.28927350489340287
                            OLS Regression Results                            
Dep. Variable:        brent_crude_oil   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6766
Date:                Mon, 04 Nov 2024   Prob (F-statistic):              0.411
Time:                        12:00:06   Log-Likelihood:                -44254.
No. Observations:                9011   AIC:                         8.851e+04
Df Residuals:                    9009   BIC:                         8.853e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------